In [1]:
# Importamos las librerías que utilizaremos
import pandas as pd
import os
import dask.dataframe as dd
import numpy as np
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service

from selenium.webdriver.chrome.options import Options
import time
from selenium.webdriver.common.by import By
import shutil
import matplotlib.pyplot as plt


In [2]:
# Definimos las carpetas de trabajo
pathRawData = r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\10. OSCE\0. Originales'
pathData = r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\10. OSCE\1. Bases'
pathResults = r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\10. OSCE\2. Resultados'

In [3]:
# Scrappeo de la web del OSCE
# Definimos algunas opciones para abrir la página
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)


# Abrimos la página
driver.maximize_window()

driver.get("https://bi.seace.gob.pe/pentaho/api/repos/%3Apublic%3Aportal%3Adatosabiertoscontratacionesdirectas.html/content?userid=public&password=key#")

# generamos un bucle para descargar todas las bases
contador = 0
for trianio in range(1,3):
    for anio in range(1,4):
        for trim in range(1,5):
            for mes in range(1,4):
                try:
                    locals()[f'{trianio}_{anio}_{trim}_{mes}'] = driver.find_element( By.XPATH, f"/html/body/main/section[1]/div[{trianio}]/div/div[{anio}]/div/div[{trim}]/a[{mes}]/button")
                    print(f'{trianio}_{anio}_{trim}_{mes}')

                    locals()[f'{trianio}_{anio}_{trim}_{mes}'].click()
                    time.sleep(5)
                    contador+=1
                except:
                    continue


1_1_1_1
1_1_1_2
1_1_1_3
1_1_2_1
1_1_2_2
1_1_2_3
1_1_3_1
1_1_3_2
1_1_3_3
1_1_4_1
1_1_4_2
1_1_4_3
1_2_1_1
1_2_1_2
1_2_1_3
1_2_2_1
1_2_2_2
1_2_2_3
1_2_3_1
1_2_3_2
1_2_3_3
1_2_4_1
1_2_4_2
1_2_4_3
1_3_1_1
1_3_1_2
1_3_1_3
1_3_2_1
1_3_2_2
1_3_2_3
1_3_3_1
1_3_3_2
1_3_3_3
1_3_4_2
1_3_4_3
2_1_1_1
2_1_1_2
2_1_1_3
2_1_2_1
2_1_2_2
2_1_2_3
2_1_3_1
2_1_3_2
2_1_3_3
2_1_4_1
2_1_4_2
2_1_4_3
2_2_1_1
2_2_1_2
2_2_1_3
2_2_2_1
2_2_2_2
2_2_2_3
2_2_3_1
2_2_3_2
2_2_3_3
2_2_4_1
2_2_4_2
2_2_4_3
2_3_1_1
2_3_1_2
2_3_1_3
2_3_2_1
2_3_2_2
2_3_2_3
2_3_3_1
2_3_3_2


In [20]:
# Seteamos la carpeta donde originalmente se descargaron los archivos
downloadPath = r"C:\Users\User\Downloads"

# Generamos una lista que permita identificar los archivos de la carpeta y las fechas de modificación
filesWithTime = [(f, os.path.getmtime(os.path.join(downloadPath, f))) for f in os.listdir(downloadPath)]

# Ordenamos de tal forma que los primeros archivos de la lista sean los más recientes
sortedFiles = sorted(filesWithTime, key=lambda x: x[1], reverse=True)

# Generamos una lista de solo los datos descargados
locals()[f'last_{contador}_files'] = [f[0] for f in sortedFiles[:contador]]

# Movemos los archivos descargados a la carpeta de trabajo
for file in locals()[f'last_{contador}_files']:

    # Path of the file to be moved
    srcFile = "C:/Users/User/Downloads/" + file

    # Destination path where the file should be moved
    pathRaw8UIT = r'C:\Users\User\OneDrive - Universidad del Pacífico\1. Documentos\0. Bases de datos\10. OSCE\0. Originales\Menores de 8 UIT'

    # Move the file from the source path to the destination path
    shutil.move(srcFile, pathRaw8UIT)


In [12]:
# Armado de la base de datos

# Generamos una lista que permita identificar los archivos de la carpeta y las fechas de modificación
pathRaw8UIT = 'C:/Users/User/OneDrive - Universidad del Pacífico/1. Documentos/0. Bases de datos/10. OSCE/0. Originales/Menores de 8 UIT/'

files_list = os.listdir(pathRaw8UIT)
os.chdir(pathData)
contador_1=0


#for file in locals()[f'last_{contador}_files']:
for file in files_list:
    if contador_1==0 :
        df = pd.read_excel(pathRaw8UIT + file, skiprows=1)
        servicios = df['TIPOORDEN'] == 'Orden de Servicio'
        df = df[servicios]
        os.remove(pathRaw8UIT + file)
        contador_1+=1
    else:
        df_alterna = pd.read_excel(pathRaw8UIT + file, skiprows=1)
        servicios = df_alterna['TIPOORDEN'] == 'Orden de Servicio'
        df_alterna = df_alterna[servicios]
        df = pd.concat([df, df_alterna])
        os.remove(pathRaw8UIT + file)
        contador_1+=1



# Guardamos la base de datos
df.to_csv('df_servicios_2018_2023.csv')


KeyboardInterrupt: 

In [23]:
ddf = dd.read_csv(pathData + '/df_servicios_2018_2023.csv')
ddf.dtypes

Unnamed: 0                         int64
ENTIDAD                           object
RUC_ENTIDAD                        int64
FECHA_REGISTRO                    object
FECHA_DE_EMISION                  object
FECHA_COMPROMISO_PRESUPUESTAL     object
FECHA_DE_NOTIFICACION             object
TIPOORDEN                         object
NRO_DE_ORDEN                       int64
ORDEN                             object
DESCRIPCION_ORDEN                 object
MONEDA                            object
MONTO_TOTAL_ORDEN_ORIGINAL       float64
OBJETOCONTRACTUAL                 object
ESTADOCONTRATACION                object
TIPODECONTRATACION                object
DEPARTAMENTO__ENTIDAD             object
RUC_CONTRATISTA                    int64
NOMBRE_RAZON_CONTRATISTA          object
dtype: object

In [24]:
ddf = ddf.loc[ : , ['ENTIDAD', 'RUC_ENTIDAD', 'FECHA_REGISTRO', 'FECHA_DE_EMISION', 'DESCRIPCION_ORDEN', 'MONEDA','MONTO_TOTAL_ORDEN_ORIGINAL', 'ESTADOCONTRATACION', 'TIPODECONTRATACION' , 'DEPARTAMENTO__ENTIDAD', 'RUC_CONTRATISTA', 'NOMBRE_RAZON_CONTRATISTA']]

In [26]:
# identificamos los ruc 10 (personas naturales)
ddf['RUC_CONTRATISTA'] = ddf['RUC_CONTRATISTA'].astype(str)
ddf['RUC_10'] = ddf['RUC_CONTRATISTA'].str.slice(0, 2)
ddfFinal = ddf[ddf['RUC_10'] == "10"]

,ENTIDAD,RUC_ENTIDAD,FECHA_REGISTRO,FECHA_DE_EMISION,DESCRIPCION_ORDEN,MONEDA,MONTO_TOTAL_ORDEN_ORIGINAL,ESTADOCONTRATACION,TIPODECONTRATACION,DEPARTAMENTO__ENTIDAD,RUC_CONTRATISTA,NOMBRE_RAZON_CONTRATISTA,RUC_10
0,ESCUELA NACIONAL DE MARINA MERCANTE,20131379430,2018-04-30,2018-03-26,SERVICIO NECESARIO PARA EL AREA DE CAPACITACION,Nuevos Soles,1800.0,Comprometida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PROV.CONSTITUC.DEL CALLAO,10458010850,LEI TASSILE MARCO ROBERTO SALOMON ...,10
1,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,20148092282,2018-04-30,2018-03-09,SERVICIOS DIVERSOS,Nuevos Soles,3701.0,Emitida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,LIMA,10460226029,GONZALES COLONIA JAVIER ERASMO ...,10
2,MUNICIPALIDAD DISTRITAL DE LA HUACA,20146697527,2018-04-30,2018-04-23,secretaria de gerencia correspondiente abril 2018,Nuevos Soles,950.0,Devengada,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PIURA,10459825172,CALLAN CRESPO GIORLLINIA LISET ...,10
3,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,20148092282,2018-04-30,2018-03-20,SERVICIOS DIVERSOS,Nuevos Soles,2000.0,Emitida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,LIMA,10457711714,HINOSTROZA CISNEROS JOHMAN ANTONIO ...,10
4,MUNICIPALIDAD DISTRITAL DE LA HUACA,20146697527,2018-04-30,2018-04-23,asistente de opi correspondiente abril 2018,Nuevos Soles,850.0,Devengada,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PIURA,10462328279,CASTILLO FLORES MARGOT ANGELITA ...,10


In [39]:
ddfFinal['FECHA_DE_EMISION'] = ddfFinal['FECHA_DE_EMISION'].astype(str)
ddfFinal['anioEmi'] = ddfFinal['FECHA_DE_EMISION'].str.slice(stop = 4)
ddfFinal['mesEmi'] = ddfFinal['FECHA_DE_EMISION'].str.slice(5, 7)
ddfFinal.head()

,ENTIDAD,RUC_ENTIDAD,FECHA_REGISTRO,FECHA_DE_EMISION,DESCRIPCION_ORDEN,MONEDA,MONTO_TOTAL_ORDEN_ORIGINAL,ESTADOCONTRATACION,TIPODECONTRATACION,DEPARTAMENTO__ENTIDAD,RUC_CONTRATISTA,NOMBRE_RAZON_CONTRATISTA,RUC_10,anioEmi,mesEmi
0,ESCUELA NACIONAL DE MARINA MERCANTE,20131379430,2018-04-30,2018-03-26,SERVICIO NECESARIO PARA EL AREA DE CAPACITACION,Nuevos Soles,1800.0,Comprometida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PROV.CONSTITUC.DEL CALLAO,10458010850,LEI TASSILE MARCO ROBERTO SALOMON ...,10,2018,03
1,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,20148092282,2018-04-30,2018-03-09,SERVICIOS DIVERSOS,Nuevos Soles,3701.0,Emitida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,LIMA,10460226029,GONZALES COLONIA JAVIER ERASMO ...,10,2018,03
2,MUNICIPALIDAD DISTRITAL DE LA HUACA,20146697527,2018-04-30,2018-04-23,secretaria de gerencia correspondiente abril 2018,Nuevos Soles,950.0,Devengada,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PIURA,10459825172,CALLAN CRESPO GIORLLINIA LISET ...,10,2018,04
3,UNIVERSIDAD NACIONAL MAYOR DE SAN MARCOS,20148092282,2018-04-30,2018-03-20,SERVICIOS DIVERSOS,Nuevos Soles,2000.0,Emitida,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,LIMA,10457711714,HINOSTROZA CISNEROS JOHMAN ANTONIO ...,10,2018,03
4,MUNICIPALIDAD DISTRITAL DE LA HUACA,20146697527,2018-04-30,2018-04-23,asistente de opi correspondiente abril 2018,Nuevos Soles,850.0,Devengada,Contrataciones hasta 8 UIT (LEY 30225)(No incl...,PIURA,10462328279,CASTILLO FLORES MARGOT ANGELITA ...,10,2018,04


In [ ]:
df = ddfFinal.compute()

In [ ]:
# Tabla N° 1: Número de órdenes de servicio anuales por entidad
tabla_1 = df.groupby( ['RUC_ENTIDAD' , 'anio_emi' ] )[['RUC_CONTRATISTA']].count().rename(columns = {'RUC_CONTRATISTA' : "n_ordenes" }).reset_index()

In [ ]:
# Tabla N° 2: Número de órdenes de servicio anuales por proveedor en cada entidad
tabla_2 = df.groupby( ['RUC_ENTIDAD' , 'anio_emi' ] )[['RUC_CONTRATISTA']].value_counts().reset_index()
tabla_2.columns = ['RUC_ENTIDAD' , 'anio_emi','RUC_CONTRATISTA','n_orden_prom']

In [ ]:
# Tabla N° 3: Monto anual contratado por órdenes de servicio por entidad
tabla_3 = df.groupby( ['RUC_ENTIDAD' , 'anio_emi' ] )[['MONTO_TOTAL_ORDEN_ORIGINAL']].sum().rename(columns = {'MONTO_TOTAL_ORDEN_ORIGINAL' : "monto_total_entidad" }).reset_index()

In [ ]:
# Tabla N° 4: Monto anual contratado por órdenes de servicio por entidad y proveedor
tabla_4 = df.groupby( ['RUC_ENTIDAD' , 'anio_emi', 'RUC_CONTRATISTA' ])[['MONTO_TOTAL_ORDEN_ORIGINAL']].sum().rename(columns = {'MONTO_TOTAL_ORDEN_ORIGINAL' : "monto_total_contratista" }).reset_index()

In [ ]:
# Tabla N° 5: Monto mensual promedio contratado por órdenes de servicio por entidad y proveedor
tabla_5 = tabla_4
tabla_5["monto_promedio_contratista_mensual"] = tabla_5["monto_total_contratista"] / 12

tabla_5 = tabla_5[tabla_5['monto_promedio_contratista_mensual'] <= 41400]

In [ ]:
# Gráfico N° 1: Distribución del monto promedio mensual en órdenes de servicio por contratista 2018-2022
    # Calculate the number of bins using the Freedman-Diaconis rule
    num_bins = int(round(len(tabla_5['monto_promedio_contratista_mensual']) ** 0.5))

    # Create the plot with journal paper format settings
    plt.figure(figsize=(8, 6))  # Adjust figure size to your preference
    plt.hist(tabla_5['monto_promedio_contratista_mensual'], bins=num_bins, color='skyblue', edgecolor='black')

    # Customize the plot
    plt.xlabel('Monto promedio mensual (en soles)', fontsize=12)  # Adjust font size
    plt.ylabel('Frequency', fontsize=12)  # Adjust font size
    plt.title('Distribución del monto promedio mensual en órdenes de servicio por contratista 2018-2022', fontsize=14)  # Adjust font size and title
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add gridlines

    # Calculate summary statistics
    mean_value = tabla_5['monto_promedio_contratista_mensual'].mean()
    median_value = tabla_5['monto_promedio_contratista_mensual'].median()
    std_value = tabla_5['monto_promedio_contratista_mensual'].std()

    # Display summary statistics on the plot
    plt.text(0.7, 0.9, f"Mean: {mean_value:.2f}", transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.7, 0.85, f"Median: {median_value:.2f}", transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.7, 0.8, f"Std: {std_value:.2f}", transform=plt.gca().transAxes, fontsize=12)

    # Save the plot (optional)
    file_path = os.path.join(pathResults,'graph1.png')
    plt.savefig(file_path, dpi=300, bbox_inches='tight')  # Adjust filename and DPI

    # Display the plot
    plt.show()

In [ ]:
# Gráfico N° 2: Distribución del número de órdenes de servicio promedio anual por contratista 2018-2022
    tabla_2 = tabla_2[tabla_2['n_orden_prom'] <= 12]
    # Calculate the number of bins using the square root rule
    num_bins = int(round(len(tabla_2['n_orden_prom']) ** 0.5))

    # Create the plot with journal paper format settings
    plt.figure(figsize=(8, 6))  # Adjust figure size to your preference
    plt.hist(tabla_2['n_orden_prom'], bins=num_bins, color='skyblue', edgecolor='black')

    # Customize the plot
    plt.xlabel('Número de órdenes promedio anual por contratista', fontsize=12)  # Adjust font size
    plt.ylabel('Frequency', fontsize=12)  # Adjust font size
    plt.title('Distribución del número de órdenes promedio anual por contratista 2018-2022', fontsize=14)  # Adjust font size and title
    plt.grid(axis='y', linestyle='--', alpha=0.7)  # Add gridlines

    # Calculate summary statistics
    mean_value = tabla_2['n_orden_prom'].mean()
    median_value = tabla_2['n_orden_prom'].median()
    std_value = tabla_2['n_orden_prom'].std()

    # Display summary statistics on the plot
    plt.text(0.7, 0.9, f"Mean: {mean_value:.2f}", transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.7, 0.85, f"Median: {median_value:.2f}", transform=plt.gca().transAxes, fontsize=12)
    plt.text(0.7, 0.8, f"Std: {std_value:.2f}", transform=plt.gca().transAxes, fontsize=12)

    # Save the plot (optional)
    file_path = os.path.join(pathResults,'graph2.png')
    plt.savefig(file_path, dpi=300, bbox_inches='tight')  # Adjust filename and DPI

    # Display the plot
    plt.show()

In [ ]:
import wordcloud as WordCloud
# Nube de palabras
word_freq = ddfFinal['DESCRIPCION_ORDEN'].str.split().explode().value_counts()
word_freq = word_freq.compute()

wordcloud = WordCloud(width=800, height=400, background_color='white').generate_from_frequencies(word_freq)

plt.figure(figsize=(10, 5))
plt.imshow(wordcloud, interpolation='bilinear')
plt.axis("off")
plt.show()